In [42]:
import numpy as np
import torch
import torch.nn as nn

## 순환 신경망(Recurrent Neural Network, RNN)

`RNN(Recurrent Neural Network)`은 뉴런의 출력이 다시 입력으로 feedback되는 `재귀적인 연결 구조`를 갖는 신경망

입력과 출력을 시퀀스 단위로 처리하는 `시퀀스(Sequence) 모델`의 일종으로 가장 기본적인 시퀀스 모델

용어는 비슷하지만 순환 신경망과 재귀 신경망(Recursive Neural Network)은 전혀 다른 개념

퍼셉트론과 같은 신경망들은 전부 은닉층에서 활성화 함수를 지난 값은 오직 출력층 방향으로 향한다

이와 같은 신경망을 `피드 포워드 신경망(Feed Forward Neural Network)`이라 한다.

이러한 신경망은 신경망은 입력이 출력 방향으로만 활성화되고 은닉 뉴런이 과거의 정보를 기억하지 못한다는 단점이 있어 입력이 들어온 문맥을 기억할 수 없다

RNN은 이러한 단점을 해결한 신경망으로 `시계열분석 문제`에 유용하다

RNN은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징을 갖고있다

$ x_t \rightarrow \text{Cell} \rightarrow y_t $

x는 입력벡터, y는 출력벡터

RNN에서 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드를 `셀(cell)`이라고 함

셀은 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행하므로 이를 `메모리 셀` 또는 `RNN 셀`이라고 표현

은닉층의 메모리 셀은 각각의 시점(time step) t 에서 바로 이전 시점 t-1 에서의 은닉층의 메모리 셀에서 나온 값 자신의 입력으로 사용

현재 시점 t 에서의 메모리 셀이 갖고있는 값은 과거의 메모리 셀들의 값에 영향을 받은 것임을 의미

메모리 셀이 출력층 방향으로 또는 다음 시점 t+1 의 자신에게 보내는 값을 `은닉 상태(hidden state)`라 함

$ \cdots \rightarrow \text{Cell}_{t-2} \rightarrow \text{Cell}_{t-1} \rightarrow \text{Cell}_{t} $

피드 포워드 신경망에서는 뉴런이라는 단위를 사용

RNN에서는 뉴런이라는 단위보다는 입력층과 출력층에서는 각각 입력 벡터와 출력 벡터, 은닉층에서는 은닉 상태라는 표현을 주로 사용

RNN은 입력과 출력의 길이를 다르게 설계 할 수 있으므로 다양한 용도로 사용가능

입출력 길이에 따라 일대다(one-to-many), 다대일(many-to-one), 다대다(many-to-many) 로 표현 가능하다.

하나의 입력에 대해서 여러개의 출력(one-to-many)의 모델의 경우 이미지 캡셔닝(Image Captioning) 작업 등에 사용 가능

시퀀스에 대해서 하나의 출력(many-to-one)을 하는 모델의 경우 입력 문서가 긍정적인지 부정적인지를 판별하는 감성 분류(sentiment classification) 작업 등에 사용 가능

다 대 다(many-to-many)의 모델의 경우 챗봇, 번역기 작업 등에 사용 가능

### RNN 수식

$ (x_t) \rightarrow W_x \rightarrow \text{Cell}_{t}(h_t) \rightarrow W_y \rightarrow (y_t) $

$ \text{Cell}_{t-1}(h_{t-1}) \rightarrow W_h \rightarrow \text{Cell}_{t}(h_t) $

입력값 $x_t$을 위한 가중치 $W_x$, 결과값 $y_t$을 위한 가중치 $W_y$, 편향 $b$

현재 시점 $t$에서 은닉 상태값 $h_t$, 은닉층의 메모리 셀 $h_t$

이전 시점 $t-1$에서 은닉 상태값 $h_{t-1}$을 위한 가중치 $W_h$

식으로 표현하면 다음과 같다

$ h_t = tanh(W_x x_t + W_h h_{y-1} + b) $

$ y_t = f(W_y h_t + b) $ 

이때 $f$는 비선형 활성화 함수

입력 벡터 $x_t$의 차원을 $d$라 하고, 은닉 상태의 크기를 $D_h$라 할때 각 벡터의 행렬 크기는 다음과 같다

$ x_t : (d \times 1) $

$ W_x : (D_h \times d) $

$ W_h : (D_h \times D_h) $

$ h_{t-1} : (D_h \times 1) $

$ h_t : (D_h \times 1) $

$ b : (D_h \times 1) $

$h_t$를 계산하기 위한 활성화 함수로는 보통 tanh 함수를 사용한다

$y_t$를 계산하기 위한 활성화 함수는 문제에 따라 다르다, 여전히 이진 분류라면 sigmoid, 다중 클래스 분류라면 softmax가 선택지가 될 수 있다

가중치 W들은 모든시점에서 값을 동일하게 공유하지만, 서로 다른 층의 가중치는 서로 다르다

### RNN 직접 구현

In [43]:
timesteps = 10      # 시점의 수
input_size = 4      # 입력의 차원
hidden_size = 8     # 은닉 상태의 크기

$ x_t : (d \times 1) $

$ x : t \times (d \times 1) $

$ h_t : (D_h \times 1) $

In [44]:
inputs = np.random.random((timesteps, input_size)) 
hidden_state_t = np.zeros((hidden_size,))
print(inputs)
print(hidden_state_t)

[[0.05528991 0.09949433 0.04031491 0.07722468]
 [0.89893192 0.90796971 0.6827166  0.51666701]
 [0.94390492 0.43345579 0.31310995 0.06104172]
 [0.00247985 0.78653874 0.28703309 0.30248925]
 [0.62293229 0.50597007 0.61388128 0.62655091]
 [0.50388334 0.74282257 0.57725823 0.46623845]
 [0.75771569 0.73868894 0.68801949 0.28156032]
 [0.62612923 0.62646282 0.04196666 0.530204  ]
 [0.51806047 0.29158857 0.02667785 0.09849487]
 [0.71488999 0.99317991 0.16114962 0.21932381]]
[0. 0. 0. 0. 0. 0. 0. 0.]


$ W_x : (D_h \times d) $

$ W_h : (D_h \times D_h) $

$ b : (D_h \times 1) $

In [45]:
Wx = np.random.random((hidden_size, input_size))
Wh = np.random.random((hidden_size, hidden_size))
b = np.random.random((hidden_size,))

$ h_t = tanh(W_x x_t + W_h h_{y-1} + b) $

In [46]:
total_hidden_states = []

# 셀 동작, 시점 수 만큼 반복
for input_t in inputs:
    output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
    total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 추적
    hidden_state_t = output_t 

    print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)


In [47]:
total_hidden_states = np.stack(total_hidden_states, axis = 0)
print(total_hidden_states) # (timesteps, output_dim)의 크기

[[0.62122651 0.73081527 0.52881886 0.58212883 0.56058841 0.49815066
  0.49640676 0.77807253]
 [0.99986682 0.99988551 0.99941602 0.99917468 0.9996889  0.9997302
  0.99995452 0.99984807]
 [0.99993971 0.999992   0.99931405 0.99948783 0.99997698 0.99997787
  0.9999969  0.99993204]
 [0.9998953  0.99999148 0.99967184 0.99940057 0.99997415 0.99991838
  0.9999964  0.99996113]
 [0.99997105 0.9999953  0.99984739 0.9997339  0.99998708 0.9999821
  0.99999896 0.99997316]
 [0.99996842 0.99999535 0.99985704 0.99976039 0.99998619 0.99997764
  0.99999871 0.99997888]
 [0.99997325 0.99999603 0.9998502  0.9998033  0.99998799 0.99998572
  0.99999883 0.99998111]
 [0.99996063 0.99999112 0.99960194 0.99957515 0.99997389 0.9999649
  0.99999726 0.99994004]
 [0.99982844 0.9999852  0.99846347 0.99865256 0.99995931 0.99993156
  0.9999931  0.99984979]
 [0.99997115 0.99999351 0.99972775 0.99975991 0.99997834 0.99997377
  0.9999974  0.99996952]]


### PyTorch로 구현

nn.RNN()을 통해서 RNN 셀을 구현가능

입력의 크기는 매 시점마다 들어가는 입력의 크기

은닉 상태의 크기는 대표적인 RNN의 하이퍼파라미터

In [48]:
input_size = 5 # 입력의 크기
hidden_size = 8 # 은닉 상태의 크기

입력 텐서는 (배치 크기 × 시점의 수 × 매 시점마다 들어가는 입력)의 크기를 가짐

In [49]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

nn.RNN()을 사용하여 RNN의 셀을 만듭니다. 인자로 입력의 크기, 은닉 상태의 크기를 정의

batch_first=True를 통해서 입력 텐서의 첫번째 차원이 배치 크기임을 명시

In [50]:
cell = nn.RNN(input_size, hidden_size, batch_first=True)
outputs, _status = cell(inputs)
print(outputs.shape) # 모든 time-step의 hidden_state, 10번의 시점동안 8차원의 은닉상태
print(_status.shape) # 최종 time-step의 hidden_state

torch.Size([1, 10, 8])
torch.Size([1, 1, 8])


## 깊은 순환 신경망(Deep Recurrent Neural Network)

RNN도 다수의 은닉층을 가질 수 있음

깊은 순환 신경망을 파이토치로 구현할 때는 nn.RNN()의 인자인 num_layers에 값을 전달하여 층을 쌓을 수 있음

In [51]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

In [52]:
# num_layers=2
cell = nn.RNN(input_size=5, hidden_size=8, num_layers=2, batch_first=True)
outputs, _status = cell(inputs)
print(outputs.shape) # 모든 time-step의 hidden_state
print(_status.shape) # 최종 time-step의 hidden_state, (층의 개수, 배치 크기, 은닉 상태의 크기)

torch.Size([1, 10, 8])
torch.Size([2, 1, 8])


## 양방향 순환 신경망(Bidirectional Recurrent Neural Network)

양방향 순환 신경망은 시점 t에서의 출력값을 예측할 때 이전 시점의 데이터뿐만 아니라, 이후 데이터로도 예측할 수 있다는 아이디어에 기반

RNN이 과거 시점(time step)의 데이터들을 참고해서, 찾고자하는 정답을 예측함

실제 문제에서는 과거 시점의 데이터만 고려하는 것이 아니라 향후 시점의 데이터에 힌트가 있는 경우도 많음

이전 시점의 데이터뿐만 아니라, `이후 시점의 데이터도 힌트로 활용`하기 위해서 고안된 것이 양방향 RNN

양방향 RNN은 하나의 출력값을 예측하기 위해 기본적으로 두 개의 메모리 셀을 사용

첫번째 메모리 셀은 앞에서 배운 것처럼 `앞 시점의 은닉 상태(Forward States)`를 전달받아 현재의 은닉 상태를 계산

두번째 메모리 셀은 앞 시점의 은닉 상태가 아니라 `뒤 시점의 은닉 상태(Backward States)`를 전달 받아 현재의 은닉 상태를 계산

다른 인공 신경망 모델들과 마찬가지로 은닉층을 무조건 추가한다고 해서 모델의 성능이 좋아지는 것은 아님

은닉층을 추가하면, 학습할 수 있는 양이 많아지지만 또한 반대로 훈련 데이터 또한 그만큼 많이 필요

구현할 때는 nn.RNN()의 인자인 bidirectional에 값을 True로 전달

In [53]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

양방향의 은닉 상태 값들이 연결(concatenate)되었기 때문에 첫번째 리턴값의 은닉 상태 크기에 * 2

두번째 리턴값의 층의 개수에 *2

정방향 기준으로는 마지막 시점에 해당되면서, 역방향 기준에서는 첫번째 시점에 해당되는 시점의 출력값을 층의 개수만큼 쌓아 올린 결과값

In [54]:
# bidirectional=True
cell = nn.RNN(input_size=5, hidden_size=8, num_layers=2, batch_first=True, bidirectional=True)
outputs, _status = cell(inputs)
print(outputs.shape) # 모든 time-step의 hidden_state, (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2)
print(_status.shape) # 최종 time-step의 hidden_state, (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)

torch.Size([1, 10, 16])
torch.Size([4, 1, 8])
